In [ ]:
# default_exp factories

# FLayout Factories
> FLayout core factories yielding native klayout datastructures.

In [ ]:
# exporti
from typing import Dict, List, Optional, Tuple, Union, overload

import pya
from flayout.cell import (
    LibReference,
    PCellLibReference,
    PCellReference,
    Reference,
    _add_cell_to_layout,
    _add_lib_cell_to_layout,
    _add_lib_pcell_to_layout,
    add_cells_to_layout,
    add_pcells_to_layout,
    reference,
)

In [ ]:
# hide
import flayout.notebook

#### Shape

In [ ]:
# exports

In [ ]:
# export
def layer(lr: int, dt: int, name: str = "") -> pya.LayerInfo:
    layer = pya.LayerInfo(int(lr), int(dt))
    if name:
        layer.name = name
    return layer

In [ ]:
layer(19, 0)

In [ ]:
# export
def point(x: Union[float, int], y: Union[float, int]) -> pya.DPoint:
    """create a KLayout DPoint

    Args:
        x: the x-value of the point
        y: the y-value of the point

    Returns:
        the point
    """
    return pya.DPoint(float(x), float(y))

In [ ]:
my_point = point(3, 5)
my_point

In [ ]:
# export
def vector(x: Union[float, int], y: Union[float, int]) -> pya.DVector:
    """create a KLayout DVector

    Args:
        x: the x-value of the vector
        y: the y-value of the vector

    Returns:
        the vector
    """
    return pya.DVector(float(x), float(y))

In [ ]:
my_vector = vector(3, 5)
my_vector

In [ ]:
# export

def _validate_point(p):
    if isinstance(p, pya.Point):
        p = p.to_dtype()
    if not isinstance(p, pya.DPoint):
        p = pya.DPoint(*(float(x) for x in p))
    return p

def box(p1: Union[pya.Point, pya.DPoint], p2: Union[pya.Point, pya.DPoint]) -> pya.DBox:
    p1 = _validate_point(p1)
    p2 = _validate_point(p2)
    return pya.DBox(p1, p2)

In [ ]:
my_box = box((0, 0), (5, 10))
my_box

In [ ]:
# export

Point = Union[pya.DPoint, Tuple[int, int], Tuple[float, float], 'np.ndarray']
Points = Union[List[Point], 'np.ndarray']

@overload
def polygon(
    hull: Points,
    *,
    holes: None = None,
    raw: bool = False,
) -> pya.DSimplePolygon:
    ...


@overload
def polygon(
    hull: Points,
    *,
    holes: List[Points] = None,
    raw: bool = False,
) -> pya.DPolygon:
    ...


def polygon(
    hull: Points,
    *,
    holes: Optional[List[Points]] = None,
    raw: bool = False,
) -> Union[pya.DSimplePolygon, pya.DPolygon]:
    """create a KLayout polygon

    Args:
        hull: the points that make the polygon hull
        holes: the collection of points that make the polygon holes
        raw: if true, the points won't be compressed.

    Returns:
        the polygon
    """
    hull = [(p if isinstance(p, (pya.Point, pya.DPoint)) else point(*p)) for p in hull]
    if holes is None:
        poly = pya.DSimplePolygon(hull, raw)
    else:
        poly = pya.DPolygon(hull, raw)
        for hole in holes:
            hole = [(p if isinstance(p, pya.DPoint) else point(*p)) for p in hole]
            poly.insert_hole(hole)
    return poly

A simple polygon is a polygon without holes. You can create this by using `fl.polygon` and only specifying a hull.

In [ ]:
my_simple_poly = polygon(
    hull=[(0, -2), (5, -2), (1, 2)],
)
my_simple_poly

General Polygons (with holes) can be specified by also specifying a `holes` keyword in `fl.polygon`:

In [ ]:
my_poly = polygon(
    hull=[(0, 0), (6, 0), (6, 4), (0, 3)],
    holes=[[(3, 1), (4, 1), (4, 2), (3, 2)]]
)
my_poly

In [ ]:
# export
def path(
    pts: Points,
    width: float,
    *,
    bgn_ext: float = 0.0,
    end_ext: float = 0.0,
    round: bool = False,
) -> pya.DPath:
    """create a KLayout path

    Args:
        pts: the points that make the path
        width: the width of the path
        bgn_ext: the begin extension of the path
        end_ext: the end extension of the path
        round: round the ends of the path

    Returns:
        the path
    """
    pts = [(p if isinstance(p, (pya.Point, pya.DPoint)) else point(*p)) for p in pts]
    print(pts)
    return pya.DPath(pts, width, bgn_ext, end_ext, round)

In [ ]:
my_path = path([(3.0, 4.0), (6.0, -1.0)], 1.2)
my_path

In [ ]:
# export
Shape = Union[pya.DBox, pya.DSimplePolygon, pya.DPolygon, pya.DPath, pya.Box, pya.SimplePolygon, pya.Polygon, pya.Path]
def cell(
    name: str,
    *,
    shapes: Optional[Dict[pya.LayerInfo, Shape]] = None,
    child_cells: Optional[List[Union[pya.Cell, Tuple[pya.Cell, pya.CplxTrans]]]] = None,
) -> pya.Cell:
    """create a KLayout cell

    Args:
        name: the name of the cell
        shapes: the shapes to add to the cell
        child_cells: the child cells to add to the cell

    Returns:
        the cell
    """
    layout = pya.Layout()
    cell = layout.create_cell(name)
    cell._layout = layout  # make sure layout does not get destroyed
    if shapes:
        for lr, lrshapes in shapes.items():
            if not isinstance(lr, pya.LayerInfo):
                lr = layer(*lr)
            lr = layout.layer(lr)
            for i, shape in enumerate(lrshapes):
                
                # if isinstance(shape, np.ndarray):
                if type(shape).__name__ == 'ndarray': # yeah... I don't want to import numpy...
                    lrshapes[i] = polygon(shape)
                    
            for i, shape in enumerate(lrshapes):  # TODO: insert all at once?
                cell.shapes(lr).insert(shape)
    if child_cells:
        cell_idxs = {}
        for ref in child_cells:
            if isinstance(ref, (str, pya.Cell, pya.PCellDeclaration)):
                ref = reference(ref)
            else:
                ref = reference(*ref)
            if isinstance(ref, Reference):
                if ref.cell not in cell_idxs:
                    cell_idxs[ref.cell] = _add_cell_to_layout(layout, ref.cell)
                cell.insert(pya.CellInstArray(cell_idxs[ref.cell], ref.trans))
            elif isinstance(ref, LibReference):
                # don't store index in cell_idxs... Cell belongs to library and will not be copied.
                idx = _add_lib_cell_to_layout(layout, ref.lib, ref.cell)
                cell.insert(pya.CellInstArray(idx, ref.trans))
            elif isinstance(ref, PCellReference):
                raise ValueError(
                    f"One can only add pcells belonging to a library to to a new cell. "
                    f"Add a pcell reference using the following string format to "
                    f"represent the cell: '<libname>.<pcellname>'."
                )
            elif isinstance(ref, PCellLibReference):
                # don't store index in cell_idxs... PCell belongs to library and will not be copied.
                idx = _add_lib_pcell_to_layout(layout, ref.lib, ref.cell, ref.params)
                cell.insert(pya.CellInstArray(idx, ref.trans))

    return cell

In [ ]:
my_cell = cell(
    name="cell0", 
    shapes={
        (10, 0): [my_simple_poly, my_poly],
        (19, 0): [my_path],
    },
)
my_cell

Child cells can also be added to a cell:

In [ ]:
my_other_poly = polygon([(10, 0), (11, 0), (11, 0), (10, 5)])
my_other_cell = cell(
    name="cell1", 
    shapes={(0, 0): [my_other_poly]},
    child_cells=[my_cell],
)
my_other_cell

In [ ]:
# export
def transform(
    mag: float = 1.0,
    rot: float = 0.0,
    mirrx: bool = False,
    x: float = 0.0,
    y: float = 0.0,
) -> pya.CplxTrans:
    """create a KLayout Transformation

    Args:
        mag: the magnitude of the transformation
        rot: the rotation of the transformation
        mirrx: mirror over the x-axis (x=0 line).
        x: translation distance in the x-direction
        y: translation distance in the y-direction

    Returns:
        the cell reference
    """
    return pya.CplxTrans(mag, rot, mirrx, int(1000 * x), int(1000 * y))

One can apply transformations to the child cell as well:

In [ ]:
my_third_poly = polygon([(0, 0), (6, 0), (6, 4), (0, 5)])
my_third_cell = cell(
    name="cell2", 
    shapes={(0, 0): [my_third_poly]},
    child_cells=[
        (my_other_cell, transform(mirrx=True, x=12.0, y=3.0)),
    ],
)
my_third_cell

The combination of a child cell and a transformation is called an `instance` of a cell (sometimes also called a reference):

In [ ]:
inst = next(my_third_cell.each_inst()) # get the only instance
inst

In [ ]:
# export
def layout(*, cells: Optional[List[pya.Cell]] = None) -> pya.Layout:
    """create a KLayout Layout

    Args:
        cells: the cells to add to the layout

    Returns:
        the KLayout layout
    """
    layout = pya.Layout()
    layout.dbu = 0.001  # hard coded for now
    if cells:
        add_cells_to_layout(layout, cells)
    return layout

Finally, a layout is a collection of multiple cells. If a cell is not a sub cell of another cell, it will be considered a top-level cell. Moreover, one cannot have cyclic references within a layout.

Let's create a layout with two top-level cells:

In [ ]:
my_independent_cell = cell(
    name='independent_cell',
    shapes={(19, 0): [polygon([(6, 0), (10, -2), (10, 0)])]}
)
    
# even though we specify four cells for our layout, only two of them are independent
# i.e. some of the cells we are added are actually child cells of other cells
# top level cells in this case are 'my_independent_cell' and 'my_third_cell'
my_layout = layout(cells=[my_independent_cell, my_cell, my_other_cell, my_third_cell])
# my_layout = layout(cells=[my_independent_cell, my_third_cell]) # equivalent

my_layout

In [ ]:
# export
def library(
    name: str,
    *,
    cells: Optional[List[pya.Cell]] = None,
    pcells: Optional[List[pya.Cell]] = None,
    description: str = "",
) -> pya.Library:
    """create a KLayout Library

    Args:
        name: the name of the library
        cells: the cells to add to the library
        pcells: the pcells to add to the library

    Returns:
        the KLayout library
    """
    lib = pya.Library()
    if description:
        lib.description = description
    layout = lib.layout()
    layout.dbu = 0.001  # hard coded for now
    if cells:
        add_cells_to_layout(layout, cells)
    if pcells:
        add_pcells_to_layout(layout, pcells)
    lib.register(name)
    return lib